<h4>Wojciech Poniewierka 402224 EAIiIB AiR AGH</h4>

<h1>Laboratorium 3</h1>

Import potrzebnych bibliotek i nawiązanie połączenia z baza danych:

In [12]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, 
                        dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

Zadanie 1. 
<br/><br/>
Ile kategorii filmów mamy w wypożyczalni?

In [13]:
query = 'SELECT COUNT(DISTINCT name) FROM category'

df = pd.read_sql(query,con=connection)
print(df)

   count
0     16


Zadanie 2. 
<br/><br/>
Wyświetl listę kategorii w kolejności alfabetycznej.

In [14]:
query = 'SELECT DISTINCT name FROM category ORDER BY name'

df = pd.read_sql(query,con=connection)
print(df)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


Zadanie 3. 
<br/><br/>
Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [15]:
query = 'SELECT title FROM film WHERE release_year = (SELECT MIN(release_year) from film) LIMIT 1'

df = pd.read_sql(query,con=connection)
print("Najmłodszy film:")
print(df)


query = 'SELECT title FROM film WHERE release_year = (SELECT MAX(release_year) from film) LIMIT 1'

df = pd.read_sql(query,con=connection)
print('\nNajstarszy film:')
print(df)

Najmłodszy film:
             title
0  Chamber Italian

Najstarszy film:
             title
0  Chamber Italian


Zadanie 4. 
<br/><br/>
Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [16]:
query = "SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2005-07-01 00:00:00' AND '2005-08-01 23:59:59'"

df = pd.read_sql(query,con=connection)
print(df)

   count
0   7380


Zadanie 5. 
<br/><br/>
Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [17]:
query = "SELECT COUNT(*) FROM rental WHERE rental_date BETWEEN '2010-01-01' AND '2011-02-01'"

df = pd.read_sql(query,con=connection)
print(df)

   count
0      0


Zadanie 6. 
<br/><br/>
Znajdź największą płatność wypożyczenia.

In [18]:
query = "SELECT MAX(amount) FROM payment"

df = pd.read_sql(query,con=connection)
print(df)

     max
0  11.99


Zadanie 7. 
<br/><br/>
Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [19]:
query = '''SELECT customer_id, first_name, last_name FROM customer WHERE address_id IN
            (SELECT address_id FROM address WHERE city_id IN
                (SELECT city_id FROM city WHERE country_id IN
                    (SELECT country_id FROM country WHERE country IN ('Poland', 'Nigeria', 'Bangladesh'))))'''

df = pd.read_sql(query,con=connection)
print(df)

    customer_id first_name  last_name
0            18      Carol     Garcia
1            21   Michelle      Clark
2            80    Marilyn       Ross
3           103     Gladys   Hamilton
4           128   Marjorie     Tucker
5           156     Bertha   Ferguson
6           198      Elsie     Kelley
7           232  Constance       Reid
8           250         Jo     Fowler
9           270       Leah     Curtis
10          277       Olga    Jimenez
11          280     Tracey    Barrett
12          284      Sonia    Gregory
13          286      Velma      Lucas
14          318      Brian      Wyman
15          329      Frank   Waggoner
16          332    Stephen     Qualls
17          380    Russell    Brinson
18          409     Rodney    Moeller
19          501      Ruben      Geary
20          558     Jimmie  Eggleston
21          562    Wallace      Slone
22          571    Johnnie   Chisholm
23          572     Sidney   Burleson


Zadanie 8. 
<br/><br/>
Gdzie mieszkają członkowie personelu?

In [20]:
query = '''SELECT co.country, ci.city, ad.address FROM country AS co, city AS ci, address AS ad, staff
           WHERE staff.address_id=ad.address_id 
           AND ad.city_id=ci.city_id
           AND ci.country_id=co.country_id'''

df = pd.read_sql(query,con=connection)
print(df)

     country        city               address
0     Canada  Lethbridge     23 Workhaven Lane
1  Australia   Woodridge  1411 Lillydale Drive


Zadanie 9. 
<br/><br/>
Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [21]:
query = '''SELECT COUNT(*) FROM staff WHERE address_id IN
            (SELECT address_id FROM address WHERE city_id IN
                (SELECT city_id FROM city WHERE country_id IN
                    (SELECT country_id FROM country where country in ('Argentina', 'Spain'))))'''

df = pd.read_sql(query,con=connection)
print(df)

   count
0      0


Zadanie 10. 
<br/><br/>
Jakie kategorie filmów zostały wypożyczone przez klientów?

In [22]:
query = '''SELECT DISTINCT name FROM category WHERE category_id IN
            (SELECT category_id FROM film_category WHERE film_id IN
                (SELECT film_id FROM film WHERE film_id IN
                    (SELECT film_id FROM inventory WHERE inventory_id IN
                        (SELECT inventory_id FROM rental))))'''
    
df = pd.read_sql(query,con=connection)
print(df)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


Zadanie 11. 
<br/><br/>
Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [23]:
query = '''SELECT DISTINCT category.name 
            FROM category, film_category, film, inventory, rental, staff, store, address, city, country WHERE
                category.category_id=film_category.category_id AND
                film_category.film_id=film.film_id AND
                film.film_id=inventory.film_id AND
                inventory.inventory_id=rental.inventory_id AND
                rental.staff_id=staff.staff_id AND
                staff.store_id=store.store_id AND
                store.address_id=address.address_id AND
                address.city_id=city.city_id AND
                city.country_id=country.country_id AND
                country.country='United States' '''

df = pd.read_sql(query,con=connection)
print(df)

Empty DataFrame
Columns: [name]
Index: []


Zadanie 12. 
<br/><br/>
Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley.

In [24]:
pfeiffer = "first_name='Olympia' AND last_name='Pfeiffer'"
zellweger = "first_name='Julia' AND last_name='Zellweger'"
presley = "first_name='Ellen' AND last_name='Presley'"

query = f'''SELECT title FROM film WHERE film_id IN
                (SELECT film_id FROM film_actor WHERE actor_id IN
                    (SELECT actor_id FROM actor WHERE {pfeiffer} OR {zellweger} OR {presley}))'''

df = pd.read_sql(query,con=connection)
print(df)

                title
0   Caribbean Liberty
1      Roots Remember
2        Outlaw Hanky
3        Women Dorado
4    Empire Malkovich
..                ...
64  Contact Anonymous
65    Majestic Floats
66       Deep Crusade
67   Effect Gladiator
68     Express Lonely

[69 rows x 1 columns]
